# alexnet (accuracy 91.8%)
## 1. 將input data轉成 ndarray(227,227,3)
## 2. 右手的資料，將其左右翻轉 (思路: 這樣可以讓它跟左手的data角度較為一致)
## 3. 使用AlexNet進行訓練

In [1]:
#input data size
ROWS = 227
COLS = 227

In [4]:
label_dict= { 0:'free', 1:'computer', 2:'cellphone', 3:'coin', 4:'ruler', 
        5:'thermos-bottle', 6:'whiteboard-pen', 7:'whiteboard-eraser',
        8:'pen', 9:'cup', 10:'remote-control-TV', 11:'remote-control-AC',
        12:'switch', 13:'windows', 14:'fridge', 15:'cupboard', 16:'water-tap',
        17:'toy', 18:'kettle', 19:'bottle', 20:'cookie', 21:'book',
        22:'magnet', 23:'lamp-switch'}

# 建構Alexnet

In [1]:
from sklearn.utils import shuffle
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
from keras.models import Sequential  
from keras.layers import Dense,Flatten,Dropout  
from keras.layers.convolutional import Conv2D,MaxPooling2D  
from keras.utils.np_utils import to_categorical  
import numpy as np  
seed = 7  
np.random.seed(seed)  
  
model = Sequential()  
model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(227,227,3),padding='valid',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Flatten())  
model.add(Dense(4096,activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(4096,activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(24,activation='softmax'))
#model.add(Dense(1000,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 256)       884992    
__________

# 訓練過程I
#### (分輪將資料餵進去)
#### 跑10輪訓練(epoch_num)
#### 每一輪 跑10組data set (這10組data set隨機餵進去)
#### 每一組 data set, 跑3個epoch

In [18]:
#for i in range(0,10,1):
epoch_num=10
for j in range(0,epoch_num,1):
    testarr=numpy.arange(0, 10, 1)
    random.shuffle(testarr)
    
    for ii in range(0,10,1):
        i=testarr[ii]
        
        Lmystr="NewLtest"+str(i)+".npy"
        Rmystr="NewRtest"+str(i)+".npy"
        LX_test=np.load(Lmystr)
        RX_test=np.load(Rmystr)
        x_img_test=np.concatenate((LX_test, RX_test), axis=0)

        Lmystr2="NewLtrain"+str(i)+".npy"
        Rmystr2="NewRtrain"+str(i)+".npy"
        LX_train=np.load(Lmystr2)
        RX_train=np.load(Rmystr2)
        x_img_train=np.concatenate((LX_train, RX_train), axis=0)

        Ltempx=np.load(Lobj_label_DATA_DIR[i])    
        Rtempx=np.load(Robj_label_DATA_DIR[i])
        y_label_train=np.concatenate((Ltempx, Rtempx), axis=0)

        TLtempx=np.load(TLobj_label_DATA_DIR[i])    
        TRtempx=np.load(TRobj_label_DATA_DIR[i])
        y_label_test=np.concatenate((TLtempx, TRtempx), axis=0)

        #Shuffle
        x_img_train, y_label_train = shuffle(x_img_train, y_label_train, random_state=0)
        x_img_test, y_label_test = shuffle(x_img_test, y_label_test, random_state=0)

        #Image normalize
        x_img_train_normalize = x_img_train.astype('float32') / 255.0
        x_img_test_normalize = x_img_test.astype('float32') / 255.0

        #轉換label 為OneHot Encoding
        y_label_train_OneHot = np_utils.to_categorical(y_label_train, num_classes=24)
        y_label_test_OneHot = np_utils.to_categorical(y_label_test, num_classes=24)

        #print: train data/test data
        print("==========[",j,",",i,"]==========")
        print("train data:",'images:',x_img_train.shape,
              " labels:",y_label_train.shape) 
        print("test  data:",'images:',x_img_test.shape ,
              " labels:",y_label_test.shape)

        train_history=model.fit(x_img_train_normalize, y_label_train_OneHot, 
                                validation_split=0.2, epochs=3, 
                                batch_size=64, verbose=1)
        
        #score = model.evaluate(x_img_train_normalize, y_label_train_OneHot, verbose=0)
        #print('Test loss:', score[0])
        #print('Test accuracy:', score[1])
        
        #arr_num[i]=x_img_train_normalize.shape[0]       
        #arr_accuracy[i]=score[1]
        #arr_loss[i]=score[0]     

==========[ 0 , 0 ]==========
train data: images: (1662, 227, 227, 3)  labels: (1662,)
test  data: images: (1660, 227, 227, 3)  labels: (1660,)
Train on 1329 samples, validate on 333 samples
Epoch 1/3
1329/1329 [==============================] - 3s - loss: 2.0591 - acc: 0.4454 - val_loss: 2.2142 - val_acc: 0.2523
Epoch 2/3
1329/1329 [==============================] - 1s - loss: 1.7362 - acc: 0.5177 - val_loss: 1.6571 - val_acc: 0.5405
Epoch 3/3
1329/1329 [==============================] - 1s - loss: 1.6320 - acc: 0.5470 - val_loss: 1.5219 - val_acc: 0.5586
==========[ 0 , 3 ]==========
train data: images: (1002, 227, 227, 3)  labels: (1002,)
test  data: images: (1078, 227, 227, 3)  labels: (1078,)
Train on 801 samples, validate on 201 samples
Epoch 1/3
801/801 [==============================] - 1s - loss: 2.5044 - acc: 0.3658 - val_loss: 2.0497 - val_acc: 0.3632
Epoch 2/3
801/801 [==============================] - 0s - loss: 1.9079 - acc: 0.3895 - val_loss: 1.7756 - val_acc: 0.5124
Epo

915/915 [==============================] - 1s - loss: 1.2094 - acc: 0.6546 - val_loss: 0.7515 - val_acc: 0.7598
Epoch 2/3
915/915 [==============================] - 1s - loss: 0.8273 - acc: 0.7038 - val_loss: 1.0315 - val_acc: 0.6288
Epoch 3/3
915/915 [==============================] - 1s - loss: 0.6301 - acc: 0.7683 - val_loss: 1.0177 - val_acc: 0.6419
==========[ 3 , 9 ]==========
train data: images: (1302, 227, 227, 3)  labels: (1302,)
test  data: images: (1132, 227, 227, 3)  labels: (1132,)
Train on 1041 samples, validate on 261 samples
Epoch 1/3
1041/1041 [==============================] - 1s - loss: 0.7408 - acc: 0.7493 - val_loss: 0.6795 - val_acc: 0.7318
Epoch 2/3
1041/1041 [==============================] - 1s - loss: 0.5790 - acc: 0.8069 - val_loss: 0.5542 - val_acc: 0.7739
Epoch 3/3
1041/1041 [==============================] - 1s - loss: 0.4938 - acc: 0.8194 - val_loss: 0.5944 - val_acc: 0.7854
==========[ 3 , 0 ]==========
train data: images: (1662, 227, 227, 3)  labels: (1

942/942 [==============================] - 1s - loss: 0.7025 - acc: 0.7633 - val_loss: 0.4321 - val_acc: 0.8305
Epoch 2/3
942/942 [==============================] - 1s - loss: 0.3703 - acc: 0.8715 - val_loss: 0.3384 - val_acc: 0.8856
Epoch 3/3
942/942 [==============================] - 1s - loss: 0.3218 - acc: 0.8864 - val_loss: 0.5301 - val_acc: 0.8220
==========[ 5 , 7 ]==========
train data: images: (1490, 227, 227, 3)  labels: (1490,)
test  data: images: (1180, 227, 227, 3)  labels: (1180,)
Train on 1192 samples, validate on 298 samples
Epoch 1/3
1192/1192 [==============================] - 1s - loss: 0.5509 - acc: 0.8221 - val_loss: 0.3416 - val_acc: 0.8624
Epoch 2/3
1192/1192 [==============================] - 1s - loss: 0.2303 - acc: 0.9195 - val_loss: 0.2820 - val_acc: 0.9027
Epoch 3/3
1192/1192 [==============================] - 1s - loss: 0.2220 - acc: 0.9195 - val_loss: 0.5583 - val_acc: 0.8490
==========[ 6 , 3 ]==========
train data: images: (1002, 227, 227, 3)  labels: (1

1580/1580 [==============================] - 1s - loss: 0.2973 - acc: 0.9032 - val_loss: 0.4223 - val_acc: 0.8813
Epoch 2/3
1580/1580 [==============================] - 1s - loss: 0.1141 - acc: 0.9620 - val_loss: 0.3411 - val_acc: 0.9091
Epoch 3/3
1580/1580 [==============================] - 1s - loss: 0.0903 - acc: 0.9709 - val_loss: 0.2863 - val_acc: 0.9217
==========[ 8 , 3 ]==========
train data: images: (1002, 227, 227, 3)  labels: (1002,)
test  data: images: (1078, 227, 227, 3)  labels: (1078,)
Train on 801 samples, validate on 201 samples
Epoch 1/3
801/801 [==============================] - 0s - loss: 0.3731 - acc: 0.8677 - val_loss: 0.4115 - val_acc: 0.8756
Epoch 2/3
801/801 [==============================] - 0s - loss: 0.2179 - acc: 0.9226 - val_loss: 0.3563 - val_acc: 0.8806
Epoch 3/3
801/801 [==============================] - 0s - loss: 0.1422 - acc: 0.9476 - val_loss: 0.2943 - val_acc: 0.9204
==========[ 8 , 6 ]==========
train data: images: (1320, 227, 227, 3)  labels: (13

# 訓練過程II
#### (分輪將資料餵進去)
#### 跑10輪訓練(epoch_num)
#### 每一輪 跑10組data set (這10組data set隨機餵進去)
#### 每一組 data set, 跑1個epoch

In [19]:
#for i in range(0,10,1):
epoch_num=10
for j in range(0,epoch_num,1):
    testarr=numpy.arange(0, 10, 1)
    random.shuffle(testarr)
    
    for ii in range(0,10,1):
        i=testarr[ii]
        
        Lmystr="NewLtest"+str(i)+".npy"
        Rmystr="NewRtest"+str(i)+".npy"
        LX_test=np.load(Lmystr)
        RX_test=np.load(Rmystr)
        x_img_test=np.concatenate((LX_test, RX_test), axis=0)

        Lmystr2="NewLtrain"+str(i)+".npy"
        Rmystr2="NewRtrain"+str(i)+".npy"
        LX_train=np.load(Lmystr2)
        RX_train=np.load(Rmystr2)
        x_img_train=np.concatenate((LX_train, RX_train), axis=0)

        Ltempx=np.load(Lobj_label_DATA_DIR[i])    
        Rtempx=np.load(Robj_label_DATA_DIR[i])
        y_label_train=np.concatenate((Ltempx, Rtempx), axis=0)

        TLtempx=np.load(TLobj_label_DATA_DIR[i])    
        TRtempx=np.load(TRobj_label_DATA_DIR[i])
        y_label_test=np.concatenate((TLtempx, TRtempx), axis=0)

        #Shuffle
        x_img_train, y_label_train = shuffle(x_img_train, y_label_train, random_state=0)
        x_img_test, y_label_test = shuffle(x_img_test, y_label_test, random_state=0)

        #Image normalize
        x_img_train_normalize = x_img_train.astype('float32') / 255.0
        x_img_test_normalize = x_img_test.astype('float32') / 255.0

        #轉換label 為OneHot Encoding
        y_label_train_OneHot = np_utils.to_categorical(y_label_train, num_classes=24)
        y_label_test_OneHot = np_utils.to_categorical(y_label_test, num_classes=24)

        #print: train data/test data
        print("==========[",j,",",i,"]==========")
        print("train data:",'images:',x_img_train.shape,
              " labels:",y_label_train.shape) 
        print("test  data:",'images:',x_img_test.shape ,
              " labels:",y_label_test.shape)

        train_history=model.fit(x_img_train_normalize, y_label_train_OneHot, 
                                validation_split=0.2, epochs=1, 
                                batch_size=64, verbose=1)
        
        #score = model.evaluate(x_img_train_normalize, y_label_train_OneHot, verbose=0)
        #print('Test loss:', score[0])
        #print('Test accuracy:', score[1])
        
        #arr_num[i]=x_img_train_normalize.shape[0]       
        #arr_accuracy[i]=score[1]
        #arr_loss[i]=score[0]     

==========[ 0 , 7 ]==========
train data: images: (1490, 227, 227, 3)  labels: (1490,)
test  data: images: (1180, 227, 227, 3)  labels: (1180,)
Train on 1192 samples, validate on 298 samples
Epoch 1/1
1192/1192 [==============================] - 1s - loss: 0.1490 - acc: 0.9539 - val_loss: 0.2021 - val_acc: 0.9295
==========[ 0 , 0 ]==========
train data: images: (1662, 227, 227, 3)  labels: (1662,)
test  data: images: (1660, 227, 227, 3)  labels: (1660,)
Train on 1329 samples, validate on 333 samples
Epoch 1/1
1329/1329 [==============================] - 1s - loss: 0.4079 - acc: 0.8849 - val_loss: 0.2568 - val_acc: 0.9249
==========[ 0 , 3 ]==========
train data: images: (1002, 227, 227, 3)  labels: (1002,)
test  data: images: (1078, 227, 227, 3)  labels: (1078,)
Train on 801 samples, validate on 201 samples
Epoch 1/1
801/801 [==============================] - 0s - loss: 0.2785 - acc: 0.9076 - val_loss: 0.3437 - val_acc: 0.87560.90
==========[ 0 , 5 ]==========
train data: images: (146

1056/1056 [==============================] - 1s - loss: 0.1673 - acc: 0.9460 - val_loss: 0.3503 - val_acc: 0.9091
==========[ 2 , 0 ]==========
train data: images: (1662, 227, 227, 3)  labels: (1662,)
test  data: images: (1660, 227, 227, 3)  labels: (1660,)
Train on 1329 samples, validate on 333 samples
Epoch 1/1
1329/1329 [==============================] - 1s - loss: 0.1143 - acc: 0.9579 - val_loss: 0.2914 - val_acc: 0.9189
==========[ 2 , 2 ]==========
train data: images: (2458, 227, 227, 3)  labels: (2458,)
test  data: images: (1858, 227, 227, 3)  labels: (1858,)
Train on 1966 samples, validate on 492 samples
Epoch 1/1
1966/1966 [==============================] - 2s - loss: 0.2022 - acc: 0.9339 - val_loss: 0.3313 - val_acc: 0.8963
==========[ 2 , 4 ]==========
train data: images: (1178, 227, 227, 3)  labels: (1178,)
test  data: images: (1316, 227, 227, 3)  labels: (1316,)
Train on 942 samples, validate on 236 samples
Epoch 1/1
942/942 [==============================] - 1s - loss: 0.

1580/1580 [==============================] - 1s - loss: 0.0807 - acc: 0.9703 - val_loss: 0.2550 - val_acc: 0.9217
==========[ 7 , 0 ]==========
train data: images: (1662, 227, 227, 3)  labels: (1662,)
test  data: images: (1660, 227, 227, 3)  labels: (1660,)
Train on 1329 samples, validate on 333 samples
Epoch 1/1
1329/1329 [==============================] - 1s - loss: 0.0878 - acc: 0.9654 - val_loss: 0.3477 - val_acc: 0.9249
==========[ 8 , 6 ]==========
train data: images: (1320, 227, 227, 3)  labels: (1320,)
test  data: images: (1006, 227, 227, 3)  labels: (1006,)
Train on 1056 samples, validate on 264 samples
Epoch 1/1
1056/1056 [==============================] - 1s - loss: 0.0688 - acc: 0.9811 - val_loss: 0.3610 - val_acc: 0.9205
==========[ 8 , 8 ]==========
train data: images: (1144, 227, 227, 3)  labels: (1144,)
test  data: images: (838, 227, 227, 3)  labels: (838,)
Train on 915 samples, validate on 229 samples
Epoch 1/1
915/915 [==============================] - 1s - loss: 0.14

# 評估訓練結果 (accuracy=91.8%)

In [20]:
arr_accuracy=np.zeros(10)
arr_loss=np.zeros(10)
arr_num=np.zeros(10)

In [21]:
for i in range(0,10,1):
    Lmystr="NewLtest"+str(i)+".npy"
    Rmystr="NewRtest"+str(i)+".npy"
    LX_test=np.load(Lmystr)
    RX_test=np.load(Rmystr)
    x_img_test=np.concatenate((LX_test, RX_test), axis=0)

    Lmystr2="NewLtrain"+str(i)+".npy"
    Rmystr2="NewRtrain"+str(i)+".npy"
    LX_train=np.load(Lmystr2)
    RX_train=np.load(Rmystr2)
    x_img_train=np.concatenate((LX_train, RX_train), axis=0)

    Ltempx=np.load(Lobj_label_DATA_DIR[i])    
    Rtempx=np.load(Robj_label_DATA_DIR[i])
    y_label_train=np.concatenate((Ltempx, Rtempx), axis=0)

    TLtempx=np.load(TLobj_label_DATA_DIR[i])    
    TRtempx=np.load(TRobj_label_DATA_DIR[i])
    y_label_test=np.concatenate((TLtempx, TRtempx), axis=0)

    #Shuffle
    x_img_train, y_label_train = shuffle(x_img_train, y_label_train, random_state=0)
    x_img_test, y_label_test = shuffle(x_img_test, y_label_test, random_state=0)

    #Image normalize
    x_img_train_normalize = x_img_train.astype('float32') / 255.0
    x_img_test_normalize = x_img_test.astype('float32') / 255.0

    #轉換label 為OneHot Encoding
    y_label_train_OneHot = np_utils.to_categorical(y_label_train, num_classes=24)
    y_label_test_OneHot = np_utils.to_categorical(y_label_test, num_classes=24)
    
    score = model.evaluate(x_img_train_normalize, y_label_train_OneHot, verbose=0)
    arr_num[i]=x_img_train_normalize.shape[0]       
    arr_accuracy[i]=score[1]
    arr_loss[i]=score[0]

In [22]:
average_accuracy=np.average(arr_accuracy, weights=arr_num)
print("(Average) Test Accuracy=",average_accuracy)

average_loss=np.average(arr_loss, weights=arr_num)
print("(Average) Test Loss=",average_loss)

(Average) Test Accuracy= 0.918689968214
(Average) Test Loss= 0.275209411445
